In [0]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession

In [0]:
spark_df = spark.table(
    "healthcare_analytics.gold_patient_risk_profile"
)

spark_df.printSchema()

In [0]:
df = (
    spark_df
    .limit(100000)
    .toPandas()
)

df.head()

In [0]:
df.isna().sum()

df = df.fillna({
    "condition_count": 0,
    "avg_cost_per_encounter": 0,
    "total_cost": 0
})

df["invalid_cost_flag"] = df["total_cost"] < 0
df["invalid_cost_flag"].value_counts()

In [0]:
np.percentile(df["total_cost"], [25, 50, 75, 90, 95])

q1 = np.percentile(df["total_cost"], 25)
q3 = np.percentile(df["total_cost"], 75)
iqr = q3 - q1

upper = q3 + 1.5 * iqr
lower = q1 - 1.5 * iqr

df["cost_outlier_flag"] = (
    (df["total_cost"] > upper) |
    (df["total_cost"] < lower)
)

df["cost_outlier_flag"].value_counts(normalize=True) * 100

In [0]:
df["norm_total_cost"] = (
    (df["total_cost"] - df["total_cost"].mean()) /
    df["total_cost"].std()
)

df["norm_encounters"] = (
    (df["total_encounters"] - df["total_encounters"].mean()) /
    df["total_encounters"].std()
)

In [0]:
df["utilization_segment"] = pd.cut(
    df["total_encounters"],
    bins=[0, 3, 10, np.inf],
    labels=["LOW", "MEDIUM", "HIGH"]
)

df["cost_segment"] = pd.cut(
    df["total_cost"],
    bins=[0, 5000, 20000, np.inf],
    labels=["LOW", "MEDIUM", "HIGH"]
)

df["risk_segment"] = (
    df["utilization_segment"].astype(str) + "_" +
    df["cost_segment"].astype(str)
)

df["risk_segment"].value_counts()

In [0]:
df["risk_score_encounters"] = 0.4 * np.log1p(df["total_encounters"])
df["risk_score_cost"] = 0.4 * np.log1p(df["total_cost"])
df["risk_score_conditions"] = 0.2 * df["condition_count"]

df["risk_score"] = (
    df["risk_score_encounters"] +
    df["risk_score_cost"] +
    df["risk_score_conditions"]
)

df[[
    "risk_score_encounters",
    "risk_score_cost",
    "risk_score_conditions",
    "risk_score"
]].head()

In [0]:
df["risk_category"] = pd.cut(
    df["risk_score"],
    bins=[-np.inf, 2, 5, np.inf],
    labels=["LOW", "MEDIUM", "HIGH"]
)

df["risk_category"].value_counts()

In [0]:
df = df.sort_values("last_encounter_date")

df["cost_trend"] = df["total_cost"].diff()

df["cost_trend_flag"] = np.where(
    df["cost_trend"] > 0, "INCREASING", "DECREASING"
)

df["cost_trend_flag"].value_counts()

In [0]:
final_cols = [
    "PATIENT",
    "total_encounters",
    "total_cost",
    "avg_cost_per_encounter",
    "condition_count",
    "risk_score",
    "risk_category",
    "risk_segment",
    "cost_outlier_flag",
    "invalid_cost_flag",
    "cost_trend_flag"
]

final_df = df[final_cols]
final_df.head()

In [0]:
spark_final = spark.createDataFrame(final_df)

(
    spark_final
    .write
    .mode("overwrite")
    .format("delta")
    .saveAsTable(
        "healthcare_analytics.gold_patient_analytics_enriched"
    )
)